<a href="https://colab.research.google.com/github/macsyd/GPU-Computing/blob/main/A4_workingtemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the nvcc4jupyter extension for CUDA compilation in Jupyter/Colab
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

# Load the extension
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-6lse80fl
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-6lse80fl
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10733 sha256=25d7336482e00933626da974c4579936e1bcd2169b147b54a7d099795d56a4e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-e8h8sjhc/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp9h3k53d5".


In [ ]:
%%cuda_group_save -g "source" -n "error_checking.h"

// Define some error checking macros.
#define cudaErrCheck(stat) { cudaErrCheck_((stat), __FILE__, __LINE__); }
void cudaErrCheck_(cudaError_t stat, const char *file, int line) {
   if (stat != cudaSuccess) {
      fprintf(stderr, "CUDA Error: %s %s %d\n", cudaGetErrorString(stat), file, line);
   }
}

#define cublasErrCheck(stat) { cublasErrCheck_((stat), __FILE__, __LINE__); }
void cublasErrCheck_(cublasStatus_t stat, const char *file, int line) {
   if (stat != CUBLAS_STATUS_SUCCESS) {
      fprintf(stderr, "cuBLAS Error: %d %s %d\n", stat, file, line);
   }
}

#define curandErrCheck(stat) { curandErrCheck_((stat), __FILE__, __LINE__); }
void curandErrCheck_(curandStatus_t stat, const char *file, int line) {
   if (stat != CURAND_STATUS_SUCCESS) {
      fprintf(stderr, "cuRand Error: %d %s %d\n", stat, file, line);
   }
}

In [ ]:
%%cuda_group_save -g "source" -n "wmma_fp16_kernel.h"

/* THE FOLLOWING CODE TAKEN FROM THE FOLLOWING RESOURCE BY NVIDIA:
 * https://github.com/NVIDIA-developer-blog/code-samples/blob/master/posts/tensor-cores/simpleTensorCoreGEMM.cu
*/

#include <stdio.h>
#include <curand.h>
#include <cublas_v2.h>

#include <mma.h>
using namespace nvcuda;

// GEMM for half precision values (i.e., fp16)
__global__ void gemm_wmma_half(half* c, int M, int N, int K, half* a, half* b) {
    // Matrix dimensions for WMMA (16x16x16 tiles)
    const int WMMA_M = 16;
    const int WMMA_N = 16;
    const int WMMA_K = 16;

    // Leading dimensions
    int lda = M;
    int ldb = K;
    int ldc = M;

    // Tile using a 2D grid
    int warpM = (blockIdx.x * blockDim.x + threadIdx.x) / warpSize;
    int warpN = (blockIdx.y * blockDim.y + threadIdx.y);

    // Declare fragments
    wmma::fragment<wmma::matrix_a, WMMA_M, WMMA_N, WMMA_K, half, wmma::row_major> a_frag;
    wmma::fragment<wmma::matrix_b, WMMA_M, WMMA_N, WMMA_K, half, wmma::col_major> b_frag;
    wmma::fragment<wmma::accumulator, WMMA_M, WMMA_N, WMMA_K, half> acc_frag;
    wmma::fragment<wmma::accumulator, WMMA_M, WMMA_N, WMMA_K, half> c_frag;

    // Initialize accumulator to 0
    wmma::fill_fragment(acc_frag, 0.0f);

    for (int i = 0; i < K; i += WMMA_K) {
      int aRow = warpM * WMMA_M;
      int aCol = i;

      int bRow = i;
      int bCol = warpN * WMMA_N;

      // Ensure only threads within bounds execute MMAs
      if (aRow < M && aCol < K && bRow < K && bCol < N) {
         /*
         // Load the inputs
         wmma::load_matrix_sync(a_frag, a + aRow + aCol * lda, lda);
         wmma::load_matrix_sync(b_frag, b + bRow + bCol * ldb, ldb);
         */

         // Fill inputs with 1s for testing
         wmma::fill_fragment(a_frag, __float2half(1.0f));  // Fill matrix A (with 1s for testing)
         wmma::fill_fragment(b_frag, __float2half(1.0f));  // Fill matrix B (with 1s for testing)

         // Perform the matrix multiplication
         wmma::mma_sync(acc_frag, a_frag, b_frag, acc_frag);
      }
   }

    // Load in the current value of c and add this to our result
    int cRow = warpM * WMMA_M;
    int cCol = warpN * WMMA_N;

    if (cRow < M && cCol < N) {
       wmma::load_matrix_sync(c_frag, c + cRow + cCol * ldc, ldc, wmma::mem_col_major);

#pragma unroll
       for(int i=0; i < c_frag.num_elements; i++) {
          c_frag.x[i] = acc_frag.x[i] + c_frag.x[i];
       }

       // Store the output (print in main)
       wmma::store_matrix_sync(c + cRow + cCol * ldc, c_frag, ldc, wmma::mem_col_major);
    }
}

In [ ]:
%%cuda_group_save -g "source" -n "wmma_fp32_kernel.h"

/* THE FOLLOWING CODE TAKEN FROM THE FOLLOWING RESOURCE BY NVIDIA:
 * https://github.com/NVIDIA-developer-blog/code-samples/blob/master/posts/tensor-cores/simpleTensorCoreGEMM.cu
*/

#include <stdio.h>
#include <curand.h>
#include <cublas_v2.h>

#include <mma.h>
using namespace nvcuda;

// GEMM for full precision values (i.e., fp32)
__global__ void gemm_wmma_full(float* c, int M, int N, int K, float* a, float* b) {
    // Matrix dimensions for WMMA (16x16x16 tiles)
    const int WMMA_M = 16;
    const int WMMA_N = 16;
    const int WMMA_K = 16;

    // Leading dimensions
    int lda = M;
    int ldb = K;
    int ldc = M;

    // Tile using a 2D grid
    int warpM = (blockIdx.x * blockDim.x + threadIdx.x) / warpSize;
    int warpN = (blockIdx.y * blockDim.y + threadIdx.y);

    // Declare fragments
    wmma::fragment<wmma::matrix_a, WMMA_M, WMMA_N, WMMA_K, half, wmma::row_major> a_frag;
    wmma::fragment<wmma::matrix_b, WMMA_M, WMMA_N, WMMA_K, half, wmma::col_major> b_frag;
    wmma::fragment<wmma::accumulator, WMMA_M, WMMA_N, WMMA_K, float> acc_frag;
    wmma::fragment<wmma::accumulator, WMMA_M, WMMA_N, WMMA_K, float> c_frag;

    // Initialize accumulator to 0
    wmma::fill_fragment(acc_frag, 0.0f);

    for (int i = 0; i < K; i += WMMA_K) {
      int aRow = warpM * WMMA_M;
      int aCol = i;

      int bRow = i;
      int bCol = warpN * WMMA_N;

      // Ensure only threads within bounds execute MMAs
      if (aRow < M && aCol < K && bRow < K && bCol < N) {
         /*
         // Load the inputs
         wmma::load_matrix_sync(a_frag, a + aRow + aCol * lda, lda);
         wmma::load_matrix_sync(b_frag, b + bRow + bCol * ldb, ldb);
         */

         // Fill inputs with 1s for testing
         wmma::fill_fragment(a_frag, 1.0f);  // Fill matrix A (with 1s for testing)
         wmma::fill_fragment(b_frag, 1.0f);  // Fill matrix B (with 1s for testing)

         // Perform the matrix multiplication
         wmma::mma_sync(acc_frag, a_frag, b_frag, acc_frag);
      }
   }

    // Load in the current value of c and add this to our result
    int cRow = warpM * WMMA_M;
    int cCol = warpN * WMMA_N;

    if (cRow < M && cCol < N) {
       wmma::load_matrix_sync(c_frag, c + cRow + cCol * ldc, ldc, wmma::mem_col_major);

#pragma unroll
       for(int i=0; i < c_frag.num_elements; i++) {
          c_frag.x[i] = acc_frag.x[i] + c_frag.x[i];
       }

       // Store the output (print in main)
       wmma::store_matrix_sync(c + cRow + cCol * ldc, c_frag, ldc, wmma::mem_col_major);
    }
}

In [ ]:
%%cuda_group_save -g "source" -n "main.cu"

/* THE FOLLOWING CODE TAKEN FROM THE FOLLOWING RESOURCE BY NVIDIA:
 * https://github.com/NVIDIA-developer-blog/code-samples/blob/master/posts/tensor-cores/simpleTensorCoreGEMM.cu
*/

/* Copyright (c) 1993-2017, NVIDIA CORPORATION. All rights reserved.
 *
 * Redistribution and use in source and binary forms, with or without
 * modification, are permitted provided that the following conditions
 * are met:
 *  * Redistributions of source code must retain the above copyright
 *    notice, this list of conditions and the following disclaimer.
 *  * Redistributions in binary form must reproduce the above copyright
 *    notice, this list of conditions and the following disclaimer in the
 *    documentation and/or other materials provided with the distribution.
 *  * Neither the name of NVIDIA CORPORATION nor the names of its
 *    contributors may be used to endorse or promote products derived
 *    from this software without specific prior written permission.
 *
 * THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS ``AS IS'' AND ANY
 * EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
 * IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR
 * PURPOSE ARE DISCLAIMED.  IN NO EVENT SHALL THE COPYRIGHT OWNER OR
 * CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL,
 * EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO,
 * PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR
 * PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY
 * OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
 * (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
 * OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
 */

#include <stdio.h>
#include <curand.h>
#include <cublas_v2.h>

#include "error_checking.h"
#include "wmma_fp16_kernel.h"
#include "wmma_fp32_kernel.h"


#include <mma.h>
using namespace nvcuda;

// Must be multiples of 16 for wmma code to work
#define MATRIX_M 16
#define MATRIX_N 16
#define MATRIX_K 16


// The only dimensions currently supported by WMMA
const int WMMA_M = 16;
const int WMMA_N = 16;
const int WMMA_K = 16;


__global__ void convertFp32ToFp16 (half *out, float *in, int n) {
   int idx = blockDim.x * blockIdx.x + threadIdx.x;
   if (idx < n) {
      out[idx] = in[idx];
   }
}

int main(int argc, char* argv[]) {
   float *a_fp32;
   float *b_fp32;
   half *a_fp16;
   half *b_fp16;

   float *c;
   float *c_wmma;
   float *c_host_wmma;

   half *c16;
   half *c16_wmma;
   half *c16_host_wmma;

   curandGenerator_t gen;

   cudaEvent_t startWMMA;
   cudaEvent_t stopWMMA;

   cudaErrCheck(cudaEventCreate(&startWMMA));
   cudaErrCheck(cudaEventCreate(&stopWMMA));

   // Use tensor cores
   cudaErrCheck(cudaMalloc((void**)&a_fp32, MATRIX_M * MATRIX_K * sizeof(float)));
   cudaErrCheck(cudaMalloc((void**)&b_fp32, MATRIX_K * MATRIX_N * sizeof(float)));
   cudaErrCheck(cudaMalloc((void**)&a_fp16, MATRIX_M * MATRIX_K * sizeof(half)));
   cudaErrCheck(cudaMalloc((void**)&b_fp16, MATRIX_K * MATRIX_N * sizeof(half)));

   cudaErrCheck(cudaMalloc((void**)&c, MATRIX_M * MATRIX_N * sizeof(float)));
   cudaErrCheck(cudaMalloc((void**)&c_wmma, MATRIX_M * MATRIX_N * sizeof(float)));
   cudaErrCheck(cudaMalloc((void**)&c16, MATRIX_M * MATRIX_N * sizeof(half)));
   cudaErrCheck(cudaMalloc((void**)&c16_wmma, MATRIX_M * MATRIX_N * sizeof(half)));

   c_host_wmma = (float*)malloc(MATRIX_M * MATRIX_N * sizeof(float));
   c16_host_wmma = (half*)malloc(MATRIX_M * MATRIX_N * sizeof(half));

   curandErrCheck(curandCreateGenerator(&gen, CURAND_RNG_PSEUDO_DEFAULT));
   curandErrCheck(curandSetPseudoRandomGeneratorSeed(gen, 1337ULL));

   curandErrCheck(curandGenerateUniform(gen, a_fp32, MATRIX_M * MATRIX_K));
   curandErrCheck(curandGenerateUniform(gen, b_fp32, MATRIX_K * MATRIX_N));

   // curand doesn't currently support fp16 so we generate in fp32 and convert to fp16.
   convertFp32ToFp16 <<< (MATRIX_M * MATRIX_K + 255) / 256, 256 >>> (a_fp16, a_fp32, MATRIX_M * MATRIX_K);
   convertFp32ToFp16 <<< (MATRIX_K * MATRIX_N + 255) / 256, 256 >>> (b_fp16, b_fp32, MATRIX_K * MATRIX_N);

   curandErrCheck(curandGenerateUniform(gen, c, MATRIX_M * MATRIX_N));
   convertFp32ToFp16 <<< (MATRIX_M * MATRIX_K + 255) / 256, 256 >>> (c16, c, MATRIX_M * MATRIX_K);

   curandErrCheck(curandDestroyGenerator(gen));

   cudaErrCheck(cudaMemcpy(c_wmma, c, MATRIX_M * MATRIX_N * sizeof(float), cudaMemcpyDeviceToDevice));
   cudaErrCheck(cudaMemcpy(c16_wmma, c16, MATRIX_M * MATRIX_N * sizeof(half), cudaMemcpyDeviceToDevice));

   printf("\nM = %d, N = %d, K = %d.\n\n", MATRIX_M, MATRIX_N, MATRIX_K);

   // Dimensions for kernel
   dim3 gridDim;
   dim3 blockDim;
   // blockDim.x must be a multple of warpSize
   // 128x4 means we have 16 warps and a block computes a 64x64 output tile
   blockDim.x = 128;
   blockDim.y = 4;

   gridDim.x = (MATRIX_M + (WMMA_M * blockDim.x / 32 - 1)) / (WMMA_M * blockDim.x / 32);
   gridDim.y = (MATRIX_N + WMMA_N * blockDim.y - 1) / (WMMA_N * blockDim.y);

   printf("Launching our GEMM WMMA Kernel...\n");
   gemm_wmma_half<<<gridDim, blockDim>>>(c16_wmma, MATRIX_M, MATRIX_N, MATRIX_K, a_fp16, b_fp16);
   gemm_wmma_full<<<gridDim, blockDim>>>(c_wmma, MATRIX_M, MATRIX_N, MATRIX_K, a_fp32, b_fp32);
   cudaErrCheck(cudaEventRecord(stopWMMA));
   cudaErrCheck(cudaEventSynchronize(stopWMMA));
   cudaDeviceSynchronize();

   cudaErrCheck(cudaEventDestroy(startWMMA));
   cudaErrCheck(cudaEventDestroy(stopWMMA));

   cudaErrCheck(cudaMemcpy(c_host_wmma, c_wmma, MATRIX_M * MATRIX_N * sizeof(float), cudaMemcpyDeviceToHost));
   cudaErrCheck(cudaMemcpy(c16_host_wmma, c16_wmma, MATRIX_M * MATRIX_N * sizeof(half), cudaMemcpyDeviceToHost));

   // Print resulting c Matrix
   printf("Contents of c after GEMM has run:\n");
   for(int a = 0; a < MATRIX_M; a++) {
      for(int b = 0; b < MATRIX_N; b++)
        {
          printf("%f ", c_host_wmma[MATRIX_M*a + b]);
        }
      printf("\n");
   }

   printf("\n");

   // Print resulting c16 Matrix
   printf("Contents of c16 after GEMM has run:\n");
   for(int a = 0; a < MATRIX_M; a++) {
      for(int b = 0; b < MATRIX_N; b++)
        {
          printf("%f ", c16_host_wmma[MATRIX_M*a + b]);
        }
      printf("\n");
   }

   cudaErrCheck(cudaFree(a_fp32));
   cudaErrCheck(cudaFree(b_fp32));
   cudaErrCheck(cudaFree(a_fp16));
   cudaErrCheck(cudaFree(b_fp16));

   cudaErrCheck(cudaFree(c));
   cudaErrCheck(cudaFree(c_wmma));
   free(c_host_wmma);
   cudaErrCheck(cudaFree(c16));
   cudaErrCheck(cudaFree(c16_wmma));
   free(c16_host_wmma);

   cudaErrCheck(cudaDeviceReset());
   return 0;
}

In [ ]:
%cuda_group_run --group "source" --compiler-args "-O3 -g -std=c++20 -arch=sm_75 -lcublas -lcurand"


M = 16, N = 16, K = 16.

Launching our GEMM WMMA Kernel...
Contents of c after GEMM has run:
16.762325 16.204956 16.689005 16.297457 16.105873 16.241302 16.541136 16.285084 16.636999 16.936016 16.282730 16.186352 16.935562 16.656845 16.622757 16.024006 
16.779079 16.504194 16.839464 16.393778 16.609661 16.244621 16.711142 16.570696 16.747826 16.447248 16.247244 16.272766 16.762253 16.301880 16.475513 16.539602 
16.954824 16.615040 16.003706 16.578810 16.778944 16.111719 16.382700 16.980839 16.081549 16.658381 16.171062 16.156496 16.482357 16.980335 16.585537 16.762894 
16.999601 16.156195 16.413279 16.850931 16.182135 16.824768 16.365646 16.084660 16.547794 16.378235 16.673759 16.499565 16.223032 16.703112 16.617823 16.919071 
16.186605 16.094723 16.163464 16.026201 16.966654 16.206564 16.132320 16.068584 16.209089 16.438320 16.531662 16.715868 16.763878 16.361742 16.684458 16.727577 
16.733587 16.886805 16.870750 16.707085 16.059801 16.784218 16.751276 16.835516 16.907932 16.689993 1

In [ ]:
%cuda_group_run --group "source" --compiler-args "-O3 -g -std=c++20 -arch=sm_75 -lcublas -lcurand" --profiler ncu --profile --profiler-args "--section SpeedOfLight"

==PROF== Connected to process 1060 (/tmp/tmp9h3k53d5/source/cuda_exec.out)
==PROF== Profiling "generate_seed_pseudo" - 0: 0%....50%....100% - 8 passes
==PROF== Profiling "gen_sequenced" - 1: 0%....50%....100% - 8 passes
==PROF== Profiling "gen_sequenced" - 2: 0%....50%....100% - 8 passes
==PROF== Profiling "convertFp32ToFp16" - 3: 0%....50%....100% - 8 passes
==PROF== Profiling "convertFp32ToFp16" - 4: 0%....50%....100% - 8 passes
==PROF== Profiling "gen_sequenced" - 5: 0%....50%....100% - 8 passes
==PROF== Profiling "convertFp32ToFp16" - 6: 0%....50%....100% - 8 passes
==PROF== Profiling "gemm_wmma_half" - 7: 0%....50%....100% - 8 passes
==PROF== Profiling "gemm_wmma_full" - 8: 0%....50%....100% - 8 passes

M = 16, N = 16, K = 16.

Launching our GEMM WMMA Kernel...
Contents of c after GEMM has run:
16.762325 16.204956 16.689005 16.297457 16.105873 16.241302 16.541136 16.285084 16.636999 16.936016 16.282730 16.186352 16.935562 16.656845 16.622757 16.024006 
16.779079 16.504194 16.83946